In [1]:
from keras.layers import Input, Dense, Dropout, Activation
from keras.models import Sequential
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.utils import to_categorical
import pandas as pd
import numpy as np

In [25]:
df= pd.read_csv('train_embeddings.csv')
df.head()

,text,sentiment,embeddings
0,sooo sad i will miss you here in san diego!!!,0,"[0.008974676951766014, -0.043760549277067184, ..."
1,my boss is bullying me...,0,"[0.010316452942788601, -0.02224578522145748, 0..."
2,what interview! leave me alone,0,"[0.0156759824603796, 0.0006652711308561265, -0..."
3,"sons of ****, why couldn`t they put them on t...",0,"[0.06485775858163834, 0.024564877152442932, -0..."
4,2am feedings for the baby are fun when he is a...,1,"[-0.014865541830658913, -0.002915020799264312,..."


In [26]:
def parse_embedding(embedding_string):
    # Remove leading and trailing square brackets
    embedding_string = embedding_string.strip('[]')
    
    # Split the string by comma and convert each part to float
    embedding_values = [float(value) for value in embedding_string.split(',') if value.strip()]
    
    # Convert the list of floats to a NumPy array
    embedding_array = np.array(embedding_values)
    
    return embedding_array


In [29]:
X = np.stack(df['embeddings'].apply(parse_embedding))
y = df['sentiment']


In [30]:
X

array([[ 0.00897468, -0.04376055, -0.04198886, ...,  0.01069653,
         0.02216817, -0.00346032],
       [ 0.01031645, -0.02224579,  0.04267559, ..., -0.0084945 ,
        -0.05490958,  0.00380819],
       [ 0.01567598,  0.00066527, -0.00682424, ...,  0.00380464,
         0.01468696,  0.03098108],
       ...,
       [-0.03910296,  0.00651592,  0.03609275, ..., -0.01229419,
        -0.00607631, -0.00477238],
       [ 0.01765929,  0.00100825, -0.0542265 , ..., -0.00152317,
        -0.03097064,  0.0198221 ],
       [ 0.01786791, -0.00962171, -0.0439616 , ..., -0.00485511,
         0.02500418,  0.02210337]])

In [31]:
len(X[0])

1536

In [32]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)


In [49]:
model = Sequential()
model.add(Dense(512,activation='relu',input_shape=(1536,)))
model.add(Dropout(0.4))
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(32,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1,activation='sigmoid'))

opt = Adam(learning_rate=0.00001)
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
checkpoint_callback = ModelCheckpoint('best_model.keras', monitor='val_accuracy', 
                                      mode='max', save_best_only=True, verbose=1)

model.summary()


c:\Users\Aditya Joshi\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_13"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_43 (Dense)                │ (None, 512)            │       786,944 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_30 (Dropout)            │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_44 (Dense)                │ (None, 128)            │        65,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_31 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_45 (Dense)                │ (None, 32)             │         4,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_32 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_46 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 856,769 (3.27 MB)

 Trainable params: 856,769 (3.27 MB)

 Non-trainable params: 0 (0.00 B)

In [50]:
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=[X_test,y_test],callbacks=[checkpoint_callback])

Epoch 1/50
460/461 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5836 - loss: 0.6872
Epoch 1: val_accuracy improved from -inf to 0.87416, saving model to best_model.keras
461/461 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.5840 - loss: 0.6872 - val_accuracy: 0.8742 - val_loss: 0.6426
Epoch 2/50
457/461 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8522 - loss: 0.6180
Epoch 2: val_accuracy improved from 0.87416 to 0.88821, saving model to best_model.keras
461/461 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.8524 - loss: 0.6176 - val_accuracy: 0.8882 - val_loss: 0.4886
Epoch 3/50
460/461 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8947 - loss: 0.4618
Epoch 3: val_accuracy improved from 0.88821 to 0.90104, saving model to best_model.keras
461/461 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.8947 - loss: 0.4616 - val_accuracy: 0.9010 - val_loss: 0.3471
Epoch 4/50
458/461 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9060 - loss: 0.3337
Epoch 4: val_accuracy improved from 0.901

In [51]:
from tensorflow.keras.models import save_model

# Save Keras model
save_model(model, 'text_model.h5')

In [64]:
model_path = r"C:\Users\Work\Desktop\cppCode\project\model_creation\keras_model"  # Specify the filepath without an extension

# Save the model using TensorFlow SavedModel format (no file extension)
model.export(model_path)

INFO:tensorflow:Assets written to: C:\Users\Work\Desktop\cppCode\project\model_creation\keras_model\assets


INFO:tensorflow:Assets written to: C:\Users\Work\Desktop\cppCode\project\model_creation\keras_model\assets


Saved artifact at 'C:\Users\Work\Desktop\cppCode\project\model_creation\keras_model'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 1536), dtype=tf.float32, name='keras_tensor_234')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  1886939599568: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1886939595168: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1886482562768: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1886482548336: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1886482557312: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1886482549216: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1886482561536: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1886937821632: TensorSpec(shape=(), dtype=tf.resource, name=None)


In [ ]:
from tensorflow.keras.models import load_model
best_model = load_model('best_model.keras')



In [65]:
!python -m tf2onnx.convert --saved-model C:\Users\Work\Desktop\cppCode\project\model_creation\text_model --output model.onnx

2024-04-21 23:34:32.324436: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-21 23:34:33.362340: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


<frozen runpy>:128: RuntimeWarning: 'tf2onnx.convert' found in sys.modules after import of package 'tf2onnx', but prior to execution of 'tf2onnx.convert'; this may result in unpredictable behaviour
2024-04-21 23:34:38,003 - WARNING - ***IMPORTANT*** Installed protobuf is not cpp accelerated. Conversion will be extremely slow. See https://github.com/onnx/tensorflow-onnx/issues/1557
2024-04-21 23:34:38.004345

In [67]:
import onnxruntime as ort

# Load the ONNX model
model_path = r"C:\Users\Work\Desktop\cppCode\project\text_model.onnx"
session = ort.InferenceSession(model_path)



In [69]:
input_name = session.get_inputs()[0].name
output_name = session.get_outputs()[0].name
print(f'Input name, Output Name: {input_name, output_name}')


Input name, Output Name: ('keras_tensor_234', 'output_0')


In [72]:
# Prepare input data
input_data = [X_test[0]]

# Run inference
outputs = session.run([output_name], {input_name: input_data})

# Process the output
# ...


In [73]:
outputs

[array([[0.9901861]], dtype=float32)]

In [75]:
y_test

8812     1
10802    0
5397     1
13806    0
10064    1
        ..
15824    0
11882    0
1398     0
10742    0
6248     0
Name: sentiment, Length: 1637, dtype: int64